In [1]:
import os

In [2]:
pwd

'd:\\Work\\C_Drive\\deep_learning_project\\NLP_project\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'd:\\Work\\C_Drive\\deep_learning_project\\NLP_project\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path=CONFIG_FILE_PATH,
        params_file_path=PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import zipfile
from textSummarizer.utils.common import get_size
from textSummarizer.logging import logger

In [14]:
from zipfile import ZipFile
import zipfile
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """ 
        zip_file_path: str
        Extracts the zip file int the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file:
            zip_file.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2023-06-22 21:13:42,466: INFO: common]: yaml file: config\config.yaml loaded successfully]
[2023-06-22 21:13:42,472: INFO: common]: yaml file: params.yaml loaded successfully]
[2023-06-22 21:13:42,477: INFO: common]: created directory at: artifacts]
[2023-06-22 21:13:42,480: INFO: common]: created directory at: artifacts/data_ingestion]
[2023-06-22 21:13:42,489: INFO: 3160687854]: File already exists of size: ~ 7718 KB]
